In [97]:
pip install pandas openpyxl pillow numpy

Note: you may need to restart the kernel to use updated packages.


In [3]:
import pandas as pd
import openpyxl
from PIL import Image
from PIL import Image, ImageDraw, ImageChops, ImageFont
import os

In [65]:
category = "Birthdays"
department = "ABC"

df = pd.read_excel("data/Format Frame Auto.xlsx", sheet_name=category)
df

,jan_fullname,jan_shortname,jan_id_file
0,Mathew Sullivan,Mat Sullivan,m.sullivan
1,Carter Guerrero,Cart Guerrero,c.guerrero
2,Marley Dale,Marl Dale,m.dale
3,Harold Griffith,Harry Griffith,h.griffith
4,Alysha Davidson,Aly Davidson,a.davidson


In [39]:
feb_names = df.jan_shortname

In [ ]:
def make_circular(image_path):
    img = Image.open(image_path).convert("RGBA")
    width, height = img.size
    
    mask = Image.new('L', (width, height), 0)
    draw = ImageDraw.Draw(mask)
    draw.ellipse((0, 0, width, height), fill=255)
    
    img.putalpha(mask)
    return img

coord_arr_5 = [[204, 315], [839, 315], [1474, 315], [536, 681], [1165,682]]

# Load background
canvas = Image.open("backgrounds/bg1.png").convert("RGBA")

# Process person image
circular_image = make_circular("images/person-holder.jpg")
resize_image = 240
resized_image = circular_image.resize((resize_image, resize_image), Image.LANCZOS)


# Main process of 
counter = 0
while counter < len(coord_arr_5):   
    x = coord_arr_5[counter][0]
    y = coord_arr_5[counter][1]
    canvas.paste(resized_image, (x , y), resized_image)


    draw = ImageDraw.Draw(canvas)

    # Font 
    font = ImageFont.truetype("DejaVuSansMono.ttf", size=25)

    # Specify the text and its position
    text = feb_names[counter]
    y_text_offset = 300
    x_text_offset = 120
    text_position = (x + x_text_offset, y + y_text_offset)

    # Choose text color
    text_color = (0, 0, 0)

    draw.text(text_position, text, fill=text_color, font=font, anchor="mm")

    counter = counter + 1

# canvas.save("[{department}]-{category}-{counter}", "JPEG")
output_name = f"{department}-{category}-{counter}.jpeg"

# Convert to RGB to save as JPEG
canvas = canvas.convert("RGB")

# Save to folder
os.makedirs("output", exist_ok=True)
canvas.save(os.path.join("output", output_name))


TODO: 
- calculate image and text offset to make the anchor point be the middle
- read the name and image path from the excel file 
- use the appropriate canvas when of number of people 

TODO: 
- manipulate the excel file  
- combine old data with the past data 

TODO: 
- Make this into a video, save it and get the duration 
- Set this duration in the software upload and observe how it will play

TODO:
- create folder for image and file in the downloads folder for easy access 
- detect if the file is an image or excel file 
- move them in their respective folders inside the project 

### Basic ID generator

In [96]:
names = [
"Mir Hernandez",
"Mett Melendez",
"Sors Suarez",
"Bil Wlcox",
"Lu Jefferson",
"Emman Hardy",
"Jac Roborts",
"Eby Ho",
"Dev Dunn"
]
sep = [name.lower().split() for name in names]
first = [item[0][0] for item in sep]

sep1 = [name.lower().split() for name in names]
last = [item[1] for item in sep1]


df = pd.DataFrame({'first_name': first, 'last_name': last, "id":f"{first}.{last}"})
df['combined'] = df['first_name'] + "." + df['last_name']
print(df['combined'].to_string(index=False))

m.hernandez
 m.melendez
   s.suarez
    b.wlcox
l.jefferson
    e.hardy
  j.roborts
       e.ho
     d.dunn


### Creating groups

In [4]:
category = "Birthdays"
department = "ABC"

df = pd.read_excel("data/More Frame Auto.xlsx", sheet_name=category)

In [ ]:
import pandas as pd

df = pd.DataFrame({'name': data})
groups = []

while not df.empty:
    n = len(df)
    for size in range(8, 0, -1):
        if n % size == 0 or n >= size:
            group = df.iloc[:size]['name'].tolist()
            groups.append(group)
            df = df.iloc[size:].reset_index(drop=True)
            break

for group in groups:
    print(group)
    print(len(group))
    print("\n")


ValueError: If using all scalar values, you must pass an index

In [130]:
counter = 0
answer = 0
while counter < 8:
    answer = counter + 1
    counter = counter + 1
print(answer)

8


If i have a column of names with 15 elements how can I make an array wherein
if the data is divisible by 8, takes those 8 and append to arr 
if the data is is not divisible by higher number then subtract 1 and if divisble take those and put in the arr 


In [ ]:
coord_arr_8 = [[80,300],[530,300],[980,300],[1430,300],[300,700],[750,700],[1200,700],[1650,700]]
coord_arr_7 = [[160,300],[610,300],[1060,300],[1510,300],[700],[700],[700]]
coord_arr_6 = [[],[],[],[],[],[]]
coord_arr_5 = [[204, 300], [839, 300], [1474, 300], [536, 700], [1165,700]]
coord_arr_4 = [[],[],[],[]]
coord_arr_3 = [[],[],[]]
coord_arr_2 = [[],[]]
coord_arr_1 = [[]]